In [ ]:
from os.path import expanduser
import glob
import re
import pandas as pd
import os

In [ ]:
def sort_key(file):
    match = re.findall(r'(\d+)', file)
    return int(match[-1]) if match else -1

# Function to format time columns and print relevant information
def printTime(df_tmp):
    df_tmp['beginTime'] = df_tmp['Begin Time - hh:mm:ss.ms'].dt.strftime('%H:%M:%S.%f')
    df_tmp['endTime'] = df_tmp['End Time - hh:mm:ss.ms'].dt.strftime('%H:%M:%S.%f')
    print(df_tmp.loc[:, ['ID', 'Type', 'beginTime', 'endTime']])

# Convert time to seconds for easy computation
def time_to_seconds(t):
    return (t.hour * 3600) + (t.minute * 60) + t.second + (t.microsecond / 1e6)

In [ ]:
# Specified output folder
outFolder = os.path.join(home, "../data/sound ")

# Read the CSV file
data_split_df = pd.read_csv('split/deBarBaro_split.csv')
data_split_df.columns = ['ID','None','Cry','Split']

# Split the IDs based on the 'Group' column
train_IDs = data_split_df[data_split_df['Split'] == 'train']['ID'].tolist()
validation_IDs = data_split_df[data_split_df['Split'] == 'validation']['ID'].tolist()
test_IDs = data_split_df[data_split_df['Split']== 'test']['ID'].tolist()

# Part #1: Read Train Dataset (debarbaro) (only train)
train_annotation_files_debarbaro,train_audio_files_debarbaro = [],[]
validation_annotation_files_debarbaro,validation_audio_files_debarbaro = [],[]

for ID in train_IDs or ID in test_IDs:
    train_annotation_files_debarbaro += glob.glob(f"..//data/deBarbaroCry/kyunghun-10min-label/{ID}/*.csv")
    train_audio_files_debarbaro += glob.glob(f"../data/deBarbaroCry/kyunghun-10min-data/{ID}/*.wav")
for ID in validation_IDs:
    validation_annotation_files_debarbaro += glob.glob(home + f"/data/deBarbaroCry/kyunghun-10min-label/{ID}/*.csv")
    validation_audio_files_debarbaro += glob.glob(home + f"/data/deBarbaroCry/kyunghun-10min-data/{ID}/*.wav")

train_annotation_files_debarbaro.sort()
train_audio_files_debarbaro.sort()
validation_annotation_files_debarbaro.sort()
validation_audio_files_debarbaro.sort()

# Part#3: Retrain
alex_model_path = '.trained/train_alex_kyunghun_debarbaro_all_lena.h5'
svm_model_path = '.trained/svm-02.joblib'
import sys
sys.path.append('./src')
from train_alex import train_alex
from train_svm import train_svm

goRetraining = True
if goRetraining:
    C = 10000
    svm_model_path = f'.trained/svm-02_C_{C}.joblib'
    train_svm(train_audio_files_debarbaro,train_annotation_files_debarbaro,alex_model_path,svm_model_path,Cvalue=C)
